In [ ]:
import os
import requests
import re
from bs4 import BeautifulSoup
import lxml
import os.path
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,WebDriverException
from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings("ignore")
import pickle
import pandas as pd
from requests.exceptions import ProxyError,ConnectionError,Timeout,HTTPError,ChunkedEncodingError
import itertools
import multiprocessing
import Ipynb_importer
from IPpool import GetHeaders
from IPpool import Ip_pool
import random
import copy
import os
# import requests_toolbelt.adapters.appengine

# requests_toolbelt.adapters.appengine.monkeypatch()


In [ ]:
global ip_pool
ip_url = ['http://www.xicidaili.com/nn/']
test_url = "http://www.dianping.com/shop/9310903"
ip_pool= Ip_pool(ip_url).get_ip_pool(test_url)

In [ ]:
ip_pool

In [ ]:
def requests_visit_url_test(url,is_use_ip =False,timeout=5):
        global ip_pool
        global bad_ip
        #headers = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/61.0.3163.100 Chrome/61.0.3163.100 Safari/537.36"}
        headers = {'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
                    'Accept-Encoding':'gzip, deflate',
                    'Accept-Language':'zh-CN,zh;q=0.8,en;q=0.6',
                    'Cache-Control':'max-age=0',
                    'Connection':'keep-alive',
                    'Cookie':'_hc.v=f72dbe69-bcfc-a9d4-1f7f-d8daeca1ad7f.1505634421; _lxsdk_cuid=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lxsdk=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; __mta=222713764.1505836341840.1507819135553.1507819166825.4; wed_user_path=163|0; Hm_lvt_dbeeb675516927da776beeb1d9802bd4=1508317369; __utma=1.1801105885.1505634423.1505636820.1508318009.3; __utmz=1.1505634423.1.1.utmcsr=google.com.hk|utmccn=(referral)|utmcmd=referral|utmcct=/; _lx_utm=utm_source%3Dgoogle%26utm_medium%3Dorganic; JSESSIONID=E0D0810114F31617ED97BC416CFDCE80; aburl=1; cy=1; cye=shanghai; _lx_utm=utm_source%3Dgoogle%26utm_medium%3Dorganic; s_ViewType=10; _lxsdk_s=15f304e1839-37c-2e4-6cf%7C%7C174',                    'Host':'www.dianping.com',
                    'If-Modified-Since':'Wed, 18 Oct 2017 08:58:29 GMT',
                    'Referer':'http://www.dianping.com/search/category/219/30/g141',
                    'Upgrade-Insecure-Requests':'1',
                    'User-Agent':random.choice(GetHeaders().user_agent_list)}

        #获取任意一个请求头
        #headers["User-Agent"] = random.choice(GetHeaders().user_agent_list)
        bad_ip = []  #用来装坏的ip
        
        #*********如果不设置ip的话**********************************
        if is_use_ip == False:
            res = requests.get(url,headers = headers,timeout=timeout)  #访问url，不设代理ip访问
            html =  res.content.decode('utf-8')
            soup  =  BeautifulSoup(html,'lxml')
            if soup.title.text == '403 Forbidden' or "有道" in soup.title.text:
                print("ip被禁止了！！！！！！！！！！！！！！！！")
                raise ConnectionError
           
        #************如果设置ip的话，则执行这一段*********************
        else:
            is_continue = True   #设置循环标志
            while is_continue:
            #整理成ip地址的格式
                try:
                    #
                    print(len(bad_ip))
                    if len(ip_pool)==len(bad_ip):
                        ip_url = ['http://www.xicidaili.com/nn/']
                        test_url = 'http://www.dianping.com/'
                        ip_pool= Ip_pool().get_ip_pool(test_url,timeout=2)
                        prox_ip = random.choice(ip_pool)
                        bad_ip = []
                        
                    #******随机生成一个ip*******
                    prox_ip = random.choice(ip_pool)
                    http = prox_ip[0]
                    ip = prox_ip[1]
                    proxies={http:ip}
                    print(proxies)
                    
                    #*******用代理ip访问********
                    res = requests.get(url,headers = headers,proxies = proxies,timeout=timeout) #设代理ip访问
                    html =  res.content.decode('utf-8')
                    soup  =  BeautifulSoup(html,'lxml')
                    
                    
                    #********如果返回网页被禁止的情况，触发异常,否则返回正常**********
                    if  soup.title.text == '403 Forbidden' or "有道" in soup.title.text:
                        print("ip被禁止了!!!!!!!!!!!!!!!!!!!!")
                        raise ConnectionError
                    else:
                        is_continue = False

                except HTTPError as e:
                    bad_ip.append(prox_ip)
                    print("断网")
                    prox_ip = random.choice(ip_pool)
                    time.sleep(2)


                except Timeout as e:
                    bad_ip.append(prox_ip)
                    print("超时")
                    prox_ip = random.choice(ip_pool)
                    time.sleep(2)


                except ConnectionError as e:
                    bad_ip.append(prox_ip)
                    print("访问被拒")
                    prox_ip = random.choice(ip_pool)
                    time.sleep(2)
                    
                except AttributeError as e:
                    bad_ip.append(prox_ip)
                    print("----------")
                    prox_ip = random.choice(ip_pool)
                    time.sleep(2)

        return soup

In [ ]:
#**************将装机地址变成一组一组*****************
def get_chunk(adr_ls,n):
    new_adr_ls = copy.deepcopy(adr_ls)
    adr_chunk = [new_adr_ls[i:i + n] for i in range(0, len(new_adr_ls), n)]
    return adr_chunk

In [ ]:

#设置谷歌浏览器的参数
def set_chrome_param():
    #PROXY = '175.155.24.61:808'
    options = webdriver.ChromeOptions()
    chrome_path = '/home/zhh/chromedriver'  
    #options.add_argument('--proxy-server={0}'.format(PROXY))
    # 设置成中文
    options.add_argument('lang=zh_CN.UTF-8')
    # 添加头部
    #1允许所有图片；2阻止所有图片；3阻止第三方服务器图片
    prefs = {'profile.default_content_setting_values': { 'images':2}}
    options.add_experimental_option('prefs', prefs)
    options.add_argument('user-agent="Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.103 Safari/537.36"')
    chrome = webdriver.Chrome(chrome_path)#),chrome_options=options)
    return chrome


In [ ]:
#requests请求网页，返回一个BeautifulSoup对象
def requests_visit_url(url,prox_ip=None):
    
    #*******设置请求头*************
    headers = {"Accept":"application/json, text/javascript, */*; q=0.01",
                    "Accept-Encoding":"gzip, deflate",
                    "Accept-Language":"zh-CN,zh;q=0.8,en;q=0.6",
                    "Connection":"keep-alive",
                    "Cookie":"_hc.v=f72dbe69-bcfc-a9d4-1f7f-d8daeca1ad7f.1505634421; __utma=1.1801105885.1505634423.1505634423.1505636820.2; __utmz=1.1505634423.1.1.utmcsr=google.com.hk|utmccn=(referral)|utmcmd=referral|utmcct=/; _lxsdk_cuid=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lxsdk=15e956d9d5ec8-0a28c942cf39be-2a044871-100200-15e956d9d5fc8; _lx_utm=utm_source%3DBaidu%26utm_medium%3Dorganic; __mta=222713764.1505836341840.1507819135553.1507819166825.4; JSESSIONID=D2D3D245867334198A59E676F146C5A8; aburl=1; cy=4; cye=guangzhou; s_ViewType=10; _lxsdk_s=15f2012787a-db4-26a-c8%7C%7C47",
                    "Host":"www.dianping.com",
                    "Referer":url,
                    #"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/61.0.3163.100 Chrome/61.0.3163.100 Safari/537.36",
                    "X-Requested-With":"XMLHttpRequest"}
    headers["User-Agent"] = random.choice(GetHeaders().user_agent_list)
    #headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.103 Safari/537.36"}
    
    #********是否循环*************
    is_continue = True
    num = 0
    while is_continue:
        num +=1
        print("访问{0}次".format(num))
        try:
            #***********不设代理访问*************
            if prox_ip is None:
                res = requests.get(url,headers = headers)  #访问url，不设代理ip访问
            else:
                
            #*********设置代理ip访问*************
                http = prox_ip[0]
                ip = http+"://"+prox_ip[1]
                proxies={http:ip}
                print(proxies)
                res = requests.get(url,headers = headers,proxies = proxies) #设代理ip访问
            html =  res.content.decode('utf-8')
            soup  =  BeautifulSoup(html,'lxml')
            is_continue = False 
            
        except HTTPError as e:
            print("断网")
            time.sleep(2)
            
        except Timeout as e:
            print("请求超时")
            time.sleep(2)
            
        except ConnectionError as e:
            print("访问被拒")
            is_continue = True
            time.sleep(2)
    return soup

In [ ]:
#缓存结果
def save_cache(result,filename):
    with open(filename,'wb') as f1:
        pickle.dump(result,f1)

In [ ]:
#加载缓存
def read_cache(filename):
    with open(filename,'rb')as f1:
        result = pickle.load(f1)
    return result

In [ ]:
# 切换路径
def get_spc_path(father_path,child_path):
    os.chdir(father_path)
    if os.path.exists(child_path):
        os.chdir(child_path)
    else:
        os.mkdir(child_path)
        os.chdir(child_path)
    print("done!!")

## 1.二级分类层

In [ ]:
%%time
#获取所有的二级分类url
def classify(city_url,city,big_class):
    soup = requests_visit_url_test(city_url,is_use_ip=True)
    clf_ls = []
    for i in soup.select(".type"):
        if i.select(".name")[0].text=="星级：":
            for j in i.select(".choice-wrap")[0]:
                if hasattr(j,"NavigableString"):
                    if "不限" not in j.text.strip():
                        clf_url = "http://www.dianping.com"+j["href"] #分类网址
                        clf_tl = j.text.strip() # 分类标题
                        clf_ls.append([big_class,city,clf_tl,clf_url])
    return clf_ls

In [ ]:
clf_ls = classify("http://www.dianping.com/dongguan/hotel",'东莞',"酒店")
print("所有二级分类已经获取完毕")

In [ ]:
clf_ls

## 2.区县层

In [ ]:
def get_region(class_ls=None):
    print('正在爬取{0}'.format(class_ls[-2]))
    url = class_ls[-1]  #url
    soup = requests_visit_url_test(url,is_use_ip=True) #访问
    region_ls = [] #地区的列表
    for i in soup.select(".J_choice-trigger-wrap-downtown a"):
        if hasattr(i,"NavigableString"):
            if "不限" not in i.text.strip():
                region_url = "http://www.dianping.com" + i["href"] 
                region_tl = i.text.strip()
                region = class_ls[:-1]
                region.extend([region_tl,region_url])
                region_ls.append(region)
    return region_ls

## 3.商圈层

In [ ]:
get_region(['酒店', '广州', '经济型', 'http://www.dianping.com/guangzhou/hotel/g171'])

In [ ]:
def get_circle(region_ls=None):
    url = region_ls[-1]
    circle_ls =[]
    soup = requests_visit_url_test(url,is_use_ip=True)
    try :
        for i in soup.find_all(class_ = re.compile("recom J_choice-content-2nd $"))[0]:
            if hasattr(i,"NavigableString"):
                if "全部" not in i.text.strip():

                    circle_url = "http://www.dianping.com"+i["href"]
                    circle_tit = i.text
                    circle = region_ls[:-1] 
                    circle.extend([circle_tit,circle_url]) 
                    circle_ls.append(circle)

    #如果没有商圈的话，就拿原来的url当作是商圈的url
        if circle_ls==[]:
            print(region_ls[-2]+"没有子分类！！")
            region_ls.insert(4,"没有商圈")
            circle_ls.append(region_ls)
    except Exception as e:
        print(region_ls)
        print("*************************")

    return circle_ls
    

## 4.商家信息层

In [ ]:
#获取每个商家的链接、店名
def get_stores(circle_ls = None):
    store_ls=[] #存储的列表
    url = circle_ls[-1]  #url
    soup = requests_visit_url_test(url,is_use_ip=True,timeout=3)  #访问当前主页
    is_continue = True
    page = 0
    while is_continue:
        page += 1
        print("正在爬取第{0}页".format(page))
        
        #解析数据
        if soup.select(".J_hotel-block") ==[]:
            print("此页没有任何信息！！")
            store_ls = []
            is_continue = False 
            
        else:
            for i in soup.select(".J_hotel-block"):  
                try :
                    store_name = i.select(".hotel-name a")[0]["title"]  #酒店名
                    link = "http://www.dianping.com"+i.select(".hotel-name a")[0]["href"] #详情层的url
                except Exception as e:
                    print("get_stores函数报错了！！！")
                    print(e)
                    print("------------------------")
                    print("")
                    store_name ="出错了"
                    link = "出错了"
                store = circle_ls[:5]  #只取前5位
                store.extend([url,store_name,link]) 
                store_ls.append(store)


            #获取下一页的数据

            next_tag = soup.find_all("a",text = re.compile("下一页"))  #获取字符属性为下一页的url
            if next_tag != []:                                         #保证含有字符属性
                url = 'http://www.dianping.com'+next_tag[0]["href"]     #获取下一页的url
                print(url)
                soup = requests_visit_url_test(url,is_use_ip=True,timeout=2)   #更新BeautifulSoup对象
            else:
                url = ''
                is_continue  = False                                  #如果没有下一页，就停止更新
    
    return store_ls

In [ ]:
store_ls=get_stores(['酒店',
  '广州',
  '经济型',
  '天河区',
  '龙洞/岑村',
  'http://www.dianping.com/guangzhou/hotel/g171r2598'])

## 5.详情层

In [ ]:
#获取详情层的数据，如星级用户、浏览人数、均价
def get_info(store_ls=None):
    
    #首先先保证导入的数据不为空
    if store_ls !=[]:
        url = store_ls[-1] #详情信息表
        soup = requests_visit_url_test(url,is_use_ip =True,timeout=2)
        try:
            start = soup.select(".mid-rank-stars")[0]["title"]       #星级用户
        except Exception as e:
            print("start",e)
            start = ''

        try:
            review_count = soup.select("#reviewCount")[0].text  #浏览人数
        except Exception as e:
            review_count = ''
            print("review_coun",e)

        try:
            avgprice = soup.select("#avgPriceTitle")[0].text   #均价
        except Exception as e:
            avgprice = ''
            print("avgprice",e)

        #评分系统
        try:
            score= []
            for j in soup.select("#comment_score .item"):
                score.append(j.text)
            score = "|".join(score)

        except Exception as e:
            score = ""
            print("score",e)

        try:   
            address = soup.select(".address .item")[0].text.strip() #地址
        except Exception as e:
            address = ''
            print("address出错了",e)

        #电话号码
        try:
            tel =[]
            for i in soup.select(".tel .item"):
                tel.append(i.text)
            tel = "|".join(tel)

        except:
            tel=""
            print("tel出错了",e)

        #路径
        try:
            path = []
            for i in soup.select(".breadcrumb a"):
                path.append(i.text.strip())
            path ='>'.join(path)
        except Exception as e:
            print("path出错了",e)
            path =''
            
        #五星评论人数
        try:
            
            five_starts = soup.find_all(id=re.compile("reviewCountStar\d"))
            if five_starts ==[]:
                five =''
                four =''
                three =''
                two =''
                one =''
            else:
                five,four,three,two,one = list(map(lambda x:x.text.strip(),five_starts))
        except Exception as e:
            print("五星评论人数出错","---->",e)
            five =''
            four =''
            three =''
            two =''
            one =''

        info = store_ls[:]
        info.extend([start,review_count,avgprice,score,address,tel,five,four,three,two,one,path])
        time.sleep(1)
    return info


## 6 主程序

##### 6.1获取所有的二级分类

In [76]:
%%time

clf_ls = classify("http://www.dianping.com/shenzhen/hotel",'深圳',"酒店")
print("所有二级分类已经获取完毕")

0
{'https': 'https://42.55.170.182:80'}
所有二级分类已经获取完毕
CPU times: user 132 ms, sys: 12 ms, total: 144 ms
Wall time: 1.29 s


In [77]:
clf_ls

[['酒店', '深圳', '经济型', 'http://www.dianping.com/shenzhen/hotel/g171'],
 ['酒店', '深圳', '五星级/豪华型', 'http://www.dianping.com/shenzhen/hotel/g3020'],
 ['酒店', '深圳', '四星级/高档型', 'http://www.dianping.com/shenzhen/hotel/g3022'],
 ['酒店', '深圳', '三星级/舒适型', 'http://www.dianping.com/shenzhen/hotel/g3024'],
 ['酒店', '深圳', '精品酒店', 'http://www.dianping.com/shenzhen/hotel/g6714'],
 ['酒店', '深圳', '青年旅舍', 'http://www.dianping.com/shenzhen/hotel/g172'],
 ['酒店', '深圳', '公寓式酒店', 'http://www.dianping.com/shenzhen/hotel/g6693'],
 ['酒店', '深圳', '客栈旅舍', 'http://www.dianping.com/shenzhen/hotel/g25842'],
 ['酒店', '深圳', '度假村', 'http://www.dianping.com/shenzhen/hotel/g173'],
 ['酒店', '深圳', '更多酒店住宿', 'http://www.dianping.com/shenzhen/hotel/g174']]

#### 6.2 获取所有的行政区

In [78]:
region_ls = list(map(get_region,clf_ls))
print("获取所有的区域清单") 

正在爬取经济型
0
{'https': 'https://119.0.115.170:80'}
正在爬取五星级/豪华型
0
{'https': 'https://60.208.44.228:80'}
正在爬取四星级/高档型
0
{'https': 'https://123.185.129.59:8080'}
正在爬取三星级/舒适型
0
{'https': 'https://101.26.110.241:8118'}
正在爬取精品酒店
0
{'https': 'https://125.47.109.42:8118'}
正在爬取青年旅舍
0
{'https': 'https://123.185.129.59:8080'}
正在爬取公寓式酒店
0
{'https': 'https://182.90.110.144:8123'}
正在爬取客栈旅舍
0
{'https': 'https://171.38.67.217:8123'}
正在爬取度假村
0
{'http': 'http://116.248.165.52:80'}
访问被拒
1
{'https': 'https://120.78.73.169:808'}
正在爬取更多酒店住宿
0
{'https': 'https://153.34.199.194:80'}
获取所有的区域清单


In [79]:
#解链
region_ls = list(itertools.chain.from_iterable(region_ls))
region_ls

[['酒店', '深圳', '经济型', '福田区', 'http://www.dianping.com/shenzhen/hotel/g171r29'],
 ['酒店', '深圳', '经济型', '罗湖区', 'http://www.dianping.com/shenzhen/hotel/g171r30'],
 ['酒店', '深圳', '经济型', '南山区', 'http://www.dianping.com/shenzhen/hotel/g171r31'],
 ['酒店', '深圳', '经济型', '盐田区', 'http://www.dianping.com/shenzhen/hotel/g171r32'],
 ['酒店', '深圳', '经济型', '宝安区', 'http://www.dianping.com/shenzhen/hotel/g171r33'],
 ['酒店', '深圳', '经济型', '龙岗区', 'http://www.dianping.com/shenzhen/hotel/g171r34'],
 ['酒店',
  '深圳',
  '经济型',
  '龙华新区',
  'http://www.dianping.com/shenzhen/hotel/g171r12033'],
 ['酒店',
  '深圳',
  '经济型',
  '光明新区',
  'http://www.dianping.com/shenzhen/hotel/g171r12034'],
 ['酒店',
  '深圳',
  '经济型',
  '坪山新区',
  'http://www.dianping.com/shenzhen/hotel/g171r12035'],
 ['酒店',
  '深圳',
  '经济型',
  '大鹏新区',
  'http://www.dianping.com/shenzhen/hotel/g171r12036'],
 ['酒店',
  '深圳',
  '五星级/豪华型',
  '福田区',
  'http://www.dianping.com/shenzhen/hotel/g3020r29'],
 ['酒店',
  '深圳',
  '五星级/豪华型',
  '罗湖区',
  'http://www.dianping.com/shenz

#### 6.3 获取所有的商圈信息

In [80]:
%%time
#***************获取所有的商圈信息****************************888
circle_ls = list(map(get_circle,region_ls))

0
{'https': 'https://183.63.101.62:53281'}
0
{'https': 'https://60.208.44.228:80'}
0
{'https': 'https://112.234.181.191:808'}
0
{'https': 'https://36.25.57.60:26492'}
0
{'https': 'https://120.78.73.169:808'}
0
{'https': 'https://182.90.110.144:8123'}
0
{'https': 'https://183.63.101.62:53281'}
0
{'http': 'http://118.114.77.47:8080'}
访问被拒
1
{'https': 'https://183.63.101.62:53281'}
0
{'https': 'https://120.78.73.169:808'}
0
{'https': 'https://171.38.67.217:8123'}
0
{'https': 'https://101.26.110.241:8118'}
0
{'https': 'https://125.47.109.42:8118'}
0
{'https': 'https://123.185.129.59:8080'}
0
{'https': 'https://42.55.170.182:80'}
0
{'https': 'https://60.163.90.184:808'}
0
{'https': 'https://60.208.44.228:80'}
0
{'https': 'https://119.0.115.170:80'}
0
{'https': 'https://110.73.50.189:8123'}
光明新区没有子分类！！
0
{'https': 'https://121.231.245.205:8118'}
0
{'https': 'https://115.46.66.135:8123'}
0
{'https': 'https://115.200.11.244:80'}
0
{'https': 'https://125.47.109.42:8118'}
0
{'https': 'https://11

In [81]:
#解链
circle_ls = list(itertools.chain.from_iterable(circle_ls))

In [82]:
circle_ls

[['酒店',
  '深圳',
  '经济型',
  '福田区',
  '华强北',
  'http://www.dianping.com/shenzhen/hotel/g171r1556'],
 ['酒店',
  '深圳',
  '经济型',
  '福田区',
  '八卦岭/园岭',
  'http://www.dianping.com/shenzhen/hotel/g171r1557'],
 ['酒店',
  '深圳',
  '经济型',
  '福田区',
  '皇岗',
  'http://www.dianping.com/shenzhen/hotel/g171r1559'],
 ['酒店',
  '深圳',
  '经济型',
  '福田区',
  '梅林',
  'http://www.dianping.com/shenzhen/hotel/g171r1560'],
 ['酒店',
  '深圳',
  '经济型',
  '福田区',
  '荔枝公园片区',
  'http://www.dianping.com/shenzhen/hotel/g171r1573'],
 ['酒店',
  '深圳',
  '经济型',
  '福田区',
  '市中心区',
  'http://www.dianping.com/shenzhen/hotel/g171r1949'],
 ['酒店',
  '深圳',
  '经济型',
  '福田区',
  '沙头',
  'http://www.dianping.com/shenzhen/hotel/g171r1950'],
 ['酒店',
  '深圳',
  '经济型',
  '福田区',
  '香蜜湖',
  'http://www.dianping.com/shenzhen/hotel/g171r1951'],
 ['酒店',
  '深圳',
  '经济型',
  '福田区',
  '华强南',
  'http://www.dianping.com/shenzhen/hotel/g171r3138'],
 ['酒店',
  '深圳',
  '经济型',
  '福田区',
  '车公庙',
  'http://www.dianping.com/shenzhen/hotel/g171r7475'],
 ['酒店',
  '深圳',


#### 6.4保存所有的商圈内容

In [83]:
get_spc_path("/home/zhh/dzdp_data/","szhotel")

done!!


In [84]:
save_cache(circle_ls,"/home/zhh/dzdp_data/szhotel/hotel_深圳_circle_ls.pickle")

#### 6.5 切块访问

In [85]:
circle_ls = read_cache("/home/zhh/dzdp_data/szhotel/hotel_深圳_circle_ls.pickle")
print(len(circle_ls))
circle_chunk = get_chunk(circle_ls,5)
len(circle_chunk)

575


115

In [86]:
offset=0
save_cache(offset,"/home/zhh/dzdp_data/szhotel/深圳_offset.pickle")

In [87]:
%%time
if __name__=='__main__':
    
    
    is_continue = True
    while is_continue:
        
        try:
            #如果偏移量已经达到了660,跳出整个大循环
            offset = read_cache("/home/zhh/dzdp_data/szhotel/深圳_offset.pickle")
            end = len(circle_chunk)
            if offset >= len(circle_chunk):
                print("整个程序已经完毕！！！")
                break
                
            #************爬取代理ip******************
            global ip_pool
            ip_url = ['http://www.xicidaili.com/nn/']
            test_url = "http://www.dianping.com/"
            ip_pool= Ip_pool(ip_url).get_ip_pool(test_url)
        
                
            #***********多进程处理*************************
            for index,chunk in enumerate(circle_chunk[offset:end]):
                offset_index = offset + index                       #偏移量+计数器
                print("正在爬第{0}chunk".format(offset_index))
                pool = multiprocessing.Pool(processes=3)           # 创建3个进程
                result = []
                for i in chunk:
                    result.append(pool.apply_async(get_stores, (i, ))) #将参数全部加进去
                pool.close()                                  # 关闭进程池，表示不能再往进程池中添加进程，需要在join之前调用
                pool.join()                                   # 等待进程池中的所有进程执行完毕
                
                #***********遍历每个进程的结果*****************
                store_ls = []
                for res in result:
                    store_ls.append(res.get())               #
                
                #***********获取每一个chunk的内容****************
                print("第{0}chunk已经爬完了".format(offset_index))
                store_ls = list(itertools.chain.from_iterable(store_ls))
                save_cache(store_ls,"/home/zhh/dzdp_data/szhotel/hotel_深圳_store_ls_{0}_chunk.pickle".format(offset_index))
                
                #********爬完直接将偏移量设为下一个*************************
                last_offset_1 = offset_index+1
                save_cache(last_offset_1,"/home/zhh/dzdp_data/szhotel/深圳_offset.pickle")
                
        #***************如果出现ChunkedEncodingError重复执行这段代码*********************     
        except ChunkedEncodingError as e:
            print("出现ChunkedEncodingError")
            last_offset_2 = offset_index
            print("chunkedEncodingError保存offset:{0}".format(last_offset_2))
            save_cache(last_offset_2,"/home/zhh/dzdp_data/szhotel/深圳_offset.pickle")
            continue
            
        #***************如果出现ChunkedEncodingError重复执行这段代码*********************      
        except KeyboardInterrupt as e:
            print("手动停止")
            last_offset_3 = offset_index
            print("KeyboardInterrupt保存现有的offset:{0}".format(last_offset_3))
            save_cache(last_offset_3,"/home/zhh/dzdp_data/szhotel/深圳_offset.pickle")
            break
            
            
#     print("***********准备爬商家的详情信息******************")
#     pool = multiprocessing.Pool(processes=3) # 创建3个进程
#     result=[]
#     for i in store_ls:
#         result.append(pool.apply_async(get_info,(i,)))
#     pool.close()
#     pool.join()
    
#     print("信息页已经爬完了！！")
#     info_ls=[]
#     for res in result:
#         info_ls.append(res.get())
#     save_cache(info_ls,"/media/zhh/东方国信/智润安排/移动互联网需求/爬虫/缓存/广州_info_test_ls.pickle")

http://www.xicidaili.com/nn/
{'Accept': 'application/json, text/javascript, */*; q=0.01', 'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'zh-CN,zh;q=0.8', 'Connection': 'keep-alive', 'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3'}
坏ip: ['http', 'http://59.40.68.162:8010']
坏ip: ['http', 'http://182.38.178.163:8118']
坏ip: ['http', 'http://117.78.37.198:8000']
坏ip: ['http', 'http://110.73.2.136:8123']
坏ip: ['http', 'http://183.130.57.59:80']
坏ip: ['http', 'http://175.163.69.34:8080']
坏ip: ['http', 'http://116.248.165.52:80']
坏ip: ['http', 'http://110.73.10.241:8123']
坏ip: ['http', 'http://121.40.42.164:8123']
坏ip: ['http', 'http://122.236.200.236:808']
坏ip: ['http', 'http://114.67.149.209:808']
坏ip: ['http', 'http://110.73.4.150:8123']
坏ip: ['http', 'http://121.31.192.60:8123']
坏ip: ['http', 'http://119.166.202.144:9999']
坏ip: ['http', 

## 爬取所有的商家信息

In [ ]:
def merge_chunk(file_dir,regrex="(.*?_store_ls_\d+_chunk.pickle)"):
    file_ls = os.listdir(file_dir)  #得出列出文件
    re_ptn = re.compile(regrex)   #正则表达式
    files = []
    result =[]
    for i in file_ls:
        if re_ptn.search(i):
            files.append(i)
    
    print("合并{0}个chunk".format(len(files)))
    for i in files:
        sub_result = read_cache(file_dir+i)
        result.extend(sub_result)
    return result

##### 佛山

In [ ]:
circle_ls = read_cache("/home/zhh/dzdp_data/szhotel/hotel_深圳_circle_ls.pickle")
print(len(circle_ls))
circle_chunk = get_chunk(circle_ls,5)
len(circle_chunk)